In [3]:
!pip install transformers
!pip install datasets
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, DefaultDataCollator
df=pd.read_csv('testing.csv', sep='§', encoding='utf8')


In [2]:
sample_test= df.sample(1000,random_state=365)
sample_test=sample_test.dropna()
text = sample_test['text'].tolist()
labels = sample_test['ups'].tolist()


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
inputs=tokenizer(text,padding='max_length',truncation=True)

In [4]:
class redditDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self,idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
eval_dataset = redditDataset(inputs,labels)

In [6]:
model = DistilBertForSequenceClassification.from_pretrained('model_sentiment', num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [7]:
def dummy_data_collector(features):
    batch = {}
    batch['input_ids'] = torch.stack([f['input_ids'] for f in features])
    batch['attention_mask'] = torch.stack([f['attention_mask'] for f in features])
    batch['labels'] = torch.stack([f['labels'] for f in features])
    
    return batch

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [8]:
training_args = TrainingArguments(
    output_dir = './sentiment_test',
    num_train_epochs = 4,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
) 
trainer = Trainer(
    data_collator=dummy_data_collector,
    model=model,
    args = training_args,
    compute_metrics=compute_metrics,
    eval_dataset=eval_dataset,
)

In [9]:

trainer.train()

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  0%|          | 10/2500 [05:58<24:49:18, 35.89s/it]

{'loss': 0.6934, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


KeyboardInterrupt: 